In [187]:
import os, glob, librosa, numpy as np, torch, json
from pathlib import Path

CACHE_DIR = (Path.cwd().parent / ".models").resolve()
CACHE_DIR.mkdir(parents=True, exist_ok=True)
os.environ["HF_HUB_DISABLE_XET"] = "1"
os.environ["HF_HOME"] = str(CACHE_DIR)
print(str(CACHE_DIR).split('/')[-3:])

['SHALLOW_FUSION_EVAL', 'SF_EVAL', '.models']


In [225]:
from datasets import Dataset
from torch.utils.data import DataLoader

from transformers import (
    WhisperProcessor, WhisperForConditionalGeneration,
    GPT2LMHeadModel, AutoTokenizer
)

SR = 16_000
BATCH_SIZE = 5
WHISPER_ID = "openai/whisper-small.en"
GPT2_ID = "cwestnedge/gpt2-small-pubmed"

MANIFEST = "../data/output/manifest.jsonl"
AUDIO_DIR = "../data/output"  

DEVICE = (
    "cuda" if torch.cuda.is_available()
    else ("mps" if torch.backends.mps.is_available() else "cpu")
)
print("Device:", DEVICE)

Device: mps


In [226]:
# fast tokenizers will show token mismatch between models and will be auto loaded when we run on colab A100 set flag to false to avoid annoyingness
processor = WhisperProcessor.from_pretrained(WHISPER_ID, cache_dir=CACHE_DIR, use_fast=False)
whisper = WhisperForConditionalGeneration.from_pretrained(WHISPER_ID, cache_dir=CACHE_DIR).to(DEVICE).eval()

gpt2_tok = AutoTokenizer.from_pretrained(GPT2_ID, cache_dir=CACHE_DIR, use_fast=False)
gpt2 = GPT2LMHeadModel.from_pretrained(GPT2_ID, cache_dir=CACHE_DIR).to(DEVICE).eval()

if gpt2_tok.pad_token is None:
    gpt2_tok.add_special_tokens({"pad_token": "<|pad|>"})
    gpt2.resize_token_embeddings(len(gpt2_tok))

PAD_ID = gpt2_tok.pad_token_id # e.g. 50257
EOS_ID = gpt2_tok.eos_token_id # 50256 (unchanged)
SHARED_VOCAB = EOS_ID + 1

print(PAD_ID, EOS_ID, SHARED_VOCAB)
# 50257 50256 50257

50257 50256 50257


In [227]:
# for i in range(SHARED_VOCAB):
#   a = processor.tokenizer.decode([i])
#   b = gpt2_tok.decode([i])
#   if a != b:
#     print(f"Token mismatch at index {i}\nwhisper token: {a}\n   gpt2 token: {b} ")

In [228]:
def build_dataset(manifest_path: str, batch_size: int, num_proc: int = 4) -> Dataset:
    with open(manifest_path, encoding="utf-8") as f:
        rows = [json.loads(line) for line in f]

    ds = Dataset.from_list(rows)

    def add_audio(batch):
        batch["audio"] = [
            librosa.load(f"{AUDIO_DIR}/{fname}", sr=SR, mono=True)[0].astype(np.float32)
            for fname in batch["file"]
        ]
        return batch

    return ds.map(add_audio, batched=True, batch_size=batch_size, num_proc=num_proc)

def encode_audio(batch):
    # batch["audio"] is List[np.ndarray], each at its natural length
    feats = processor.feature_extractor(
        batch["audio"], # for whatever reason processor doesnt support PT tensors so numpy array or list for now.
        sampling_rate=SR,
        padding="max_length",
        truncation=True, 
        max_length=processor.feature_extractor.n_samples,  # n_samples = chunk_length * sampling_rate
        return_attention_mask=True,
        return_tensors="pt" 
    )

    #  input_features : Tensor (B, T_max, 80)
    #  attention_mask : Tensor (B, T_max)
    batch["input_features"] = feats.input_features
    batch["attention_mask"] = feats.attention_mask
    return batch

ds = build_dataset(MANIFEST, batch_size=BATCH_SIZE).select(range(20))

# choosing NOT to overwrite ds with removed fields so we can eval on text field later,
# could also create a collator and pass fields we care about through, but seems like 
# too much extra code tbh, indices will still match if we dont shuffle
ds_processed = ds.map(
    encode_audio, 
    batch_size=BATCH_SIZE, 
    batched=True,
    remove_columns=['uuid', 'file', 'chunk_id', 'orig_id', 'label', 'text', 'audio']
    )

ds_processed.set_format(type="torch", columns=["input_features","attention_mask"])
loader = DataLoader(ds_processed, batch_size=BATCH_SIZE, shuffle=False)

Map: 100%|██████████| 20/20 [00:01<00:00, 17.39 examples/s]


In [112]:
# def _hyphen_ids(tok, vocab_limit):
#     # any ASCII hyphen or Unicode dash char
#     dash_chars = {"-", "-", "–", "—"}
#     return torch.tensor(
#         [i for i in range(vocab_limit)
#          if any(ch in tok.decode([i]) for ch in dash_chars)],
#         dtype=torch.long
#     )

# HYPHEN_IDS = _hyphen_ids(gpt2_tok, len(gpt2_tok))

In [229]:
from transformers import LogitsProcessor

class ShallowFusion(LogitsProcessor):
    def __init__(self, lm, fusion_exclusive, pad_id, alpha=0.3, warmup_steps=3, temperature=0.05):
        super().__init__()
        self.lm = lm.eval().requires_grad_(False)
        self.fusion_excl = fusion_exclusive  # e.g. EOS_ID = 50256
        self.pad_id = pad_id
        self.alpha = alpha
        self.warmup = warmup_steps
        self.temp = temperature
        self.step = 0
        self.alpha_scale = 0.35
        self.entropy_threshold = 1.3
    
    def reset(self):
        self.step = 0

    @torch.inference_mode()
    def __call__(self, input_ids, scores):
        w_lp = torch.log_softmax(scores, dim=-1)

        if self.step < self.warmup: 
            self.step+=1 
            return w_lp
        self.step+=1 

        # ----DYNAMIC ALPHA------
        # w_probs = w_lp.exp()
        # # Compute entropy safely: ignore zero probabilities
        # ent_contrib = torch.where(w_probs > 0,
        #                           w_probs * w_lp,
        #                           torch.zeros_like(w_lp))
        # w_entropy = -(ent_contrib.sum(dim=-1, keepdim=True))  # shape: [B,1]
        # # Smooth gating: map entropy to [alpha, alpha*alpha_scale]
        # gate = torch.sigmoid((w_entropy - self.entropy_threshold) * 2)
        # ----DYNAMIC ALPHA------

        # replace OOV tokens with pad tokens and exclude from attention scores
        # fuse [0 … EOS_ID-1], leave EOS untouched

        oob_mask = input_ids > self.fusion_excl
        filtered_ids = input_ids.masked_fill(oob_mask, self.pad_id)
        attn_mask = (filtered_ids != self.pad_id).long()

        lm_logits = self.lm(
            input_ids=filtered_ids, 
            attention_mask=attn_mask
        ).logits[:,-1,:] # only want logits for next token
        lm_lp = torch.log_softmax(lm_logits, dim=-1)[:, :self.fusion_excl]

        fused_slice = w_lp[:, :self.fusion_excl] + self.alpha * lm_lp
        fused_lp = w_lp.clone()
        fused_lp[:, :self.fusion_excl] = fused_slice
        
        # renormalize
        fused_lp -= torch.logsumexp(fused_lp, dim=-1, keepdim=True)
        
        return fused_lp # / self.temp
    
fusion_proc = ShallowFusion(
    lm=gpt2,
    fusion_exclusive=EOS_ID, # e.g. EOS_ID = 50256
    pad_id=PAD_ID,# <— 50257
    alpha=0.2,
    warmup_steps=4,
    temperature = 1.0
)


In [ ]:
from transformers import LogitsProcessorList
from tqdm import tqdm 

fused = []
refs = []

for idx, batch in enumerate(tqdm(loader, total=len(loader), desc="Decoding")):
    feats = batch['input_features'].to(DEVICE)
    masks = batch['attention_mask'].to(DEVICE)

    with torch.no_grad():
        fused_ids = whisper.generate(
            input_features=feats,
            attention_mask=masks,
            logits_processor=LogitsProcessorList([fusion_proc]),
            num_beams=4,
            do_sample=False,
            length_penalty=1.0,
        )
    decoded = processor.batch_decode(fused_ids, skip_special_tokens=True)
    fused.extend(decoded)
    fusion_proc.reset()

Decoding:  25%|██▌       | 1/4 [01:55<05:46, 115.37s/it]

In [204]:
class NoOpLogitsProcessor(LogitsProcessor):
    """
    A simple logits processor that performs no modifications to the logits.
    """
    def __call__(self, input_ids, scores):
        # Return the scores unchanged

        w_lp = torch.log_softmax(scores, dim=-1)
        return w_lp


noop_processor = NoOpLogitsProcessor()
vanilla = []

for idx, batch in enumerate(tqdm(loader, total=len(loader), desc="Decoding")):
    feats = batch['input_features'].to(DEVICE)
    masks = batch['attention_mask'].to(DEVICE)
    with torch.no_grad():
        vanilla_ids = whisper.generate(
            input_features=feats,
            attention_mask=masks,
            logits_processor=LogitsProcessorList([noop_processor]),
            num_beams=1,
            do_sample=False,
            length_penalty=1.0,
        )
    decoded = processor.batch_decode(vanilla_ids, skip_special_tokens=True)
    vanilla.extend(decoded)

Decoding: 100%|██████████| 4/4 [00:28<00:00,  7.10s/it]


In [221]:
import pandas as pd 

results_df = pd.DataFrame(
    {
        "vanilla":[i.strip() for i in vanilla], 
        "fused":[i.strip() for i in fused], 
        "reference":ds['text'],
        # "medical_terms":ds['medical_terms']
    }
)

In [222]:
from jiwer import (
    Compose,
    ToLowerCase,
    RemovePunctuation,
    RemoveMultipleSpaces,
    Strip,
    ReduceToListOfListOfWords,
    wer
)
from unidecode import unidecode
import re

# helper to handle both str and list[str]
def _map(func, x):
    return [func(t) for t in x] if isinstance(x, list) else func(x)

def remove_diacritics(x):
    return _map(unidecode, x)

def split_hyphens_and_slashes(x):
    # replace any dash or slash with a space so we never glue words together
    return _map(lambda t: re.sub(r"[-–—/]", " ", t), x)

def normalize_nums(x):
    # unify 12–16 → 12-16
    return _map(lambda t: re.sub(r"(\d)[-–—-](\d)", r"\1-\2", t), x)

transform = Compose([
    ToLowerCase(),
    remove_diacritics,
    split_hyphens_and_slashes,    # ← split first
    normalize_nums,
    RemovePunctuation(),           # now drop commas, periods, etc.
    RemoveMultipleSpaces(),
    Strip(),
    ReduceToListOfListOfWords(),   # produce [[“word”,…],…]
])

def compute_wer(ref, hyp):
    return wer(
        ref, hyp,
        reference_transform=transform,
        hypothesis_transform=transform,
    )

# rename & score
results_df = results_df.rename(columns={"gt": "reference"})
results_df["wer_base"]  = results_df.apply(
    lambda r: compute_wer(r["reference"], r["vanilla"]), axis=1
)
results_df["wer_fused"] = results_df.apply(
    lambda r: compute_wer(r["reference"], r["fused"]), axis=1
)

print("Base  WER (punct-insensitive):", results_df["wer_base"].mean())
print("Fused WER (punct-insensitive):", results_df["wer_fused"].mean())

Base  WER (punct-insensitive): 0.04469273107353603
Fused WER (punct-insensitive): 0.05052003122436559


In [223]:
results_df['diff'] = abs(results_df.wer_base - results_df.wer_fused)
top_diffs = results_df.sort_values(by='diff', ascending=False).head(10)
top_diffs

,vanilla,fused,reference,wer_base,wer_fused,diff
16,He smokes two packs a day and he has two beers...,He smokes two packs a day and he has two beers...,He smokes two packs a day and he has two beers...,0.019231,0.115385,0.096154
3,the inferior pole of the right kidney. There i...,the inferior pole of the right kidney. There i...,the inferior pole of the right kidney. There i...,0.060000,0.100000,0.040000
5,"With diverticulitis, no pneumoperatinin is ide...","With diverticulitis, no pneumoperitoneum is id...",with diverticulitis. No pneumoperitoneum is id...,0.080000,0.060000,0.020000
7,"Stomach, it was insufflated and the scope was ...","Stomach, it was inseflated and the scope was c...",stomach. It was insufflated and the scope was ...,0.000000,0.020000,0.020000
8,IV Demerol and Versed for sedation. When adequ...,IV Demerol and versed for sedation. When adequ...,IV Demerol and Versed for sedation. When adequ...,0.000000,0.020000,0.020000
18,1. Lisinopril 2. Metaprolol 3. Vitamin B12 4. ...,1. Lisinopril 2. Metaprolol 3. Vitamin B12 4. ...,1. Lisinopril. 2. Metoprolol. 3. Vitamin B12. ...,0.060000,0.040000,0.020000
1,administered. The lung bases are clear. The li...,administered. The lung bases are clear. The li...,administered. The lung bases are clear. The li...,0.058824,0.039216,0.019608
0,"Gidney, adrenal, abdomen and pelvis, CT scan, ...","Gidney, adrenal, abdomen and pelvis, CT scan, ...","kidney, adrenal, abdomen and pelvis, ct scan, ...",0.111111,0.111111,0.000000
13,"bumped his head once, but not his chest, altho...","bumped his head once, but not his chest, altho...","bumped his head once, but not his chest, altho...",0.000000,0.000000,0.000000
17,"chest pain, SOB, supine, palpitations, edema, ...","chest pain, S.O.B., supine, palpitations, edem...","chest pain, SOB supine, palpitations, edema, v...",0.000000,0.000000,0.000000


In [224]:
print_str = '''
GT:    {}
Base:  {}
Fused: {}'''

for idx, row in results_df.iterrows():
    row_str = print_str.format(
        row['reference'], 
        row['vanilla'], 
        row['fused']
    )
    print(row_str)


GT:    kidney, adrenal, abdomen and pelvis, ct scan, intravenous, abdomen,
Base:  Gidney, adrenal, abdomen and pelvis, CT scan, intravenous, abdomen.
Fused: Gidney, adrenal, abdomen and pelvis, CT scan, intravenous, abdomen.

GT:    administered. The lung bases are clear. The liver is enlarged and decreased in attenuation. There are no focal liver masses. There is no intra or extrahepatic ductal dilatation. The gallbladder is slightly distended. The adrenal glands, pancreas, spleen, and left kidney are normal. A 12-mm simple cyst is present in
Base:  administered. The lung bases are clear. The liver is enlarged and decreased in attenuation. There are no focal liver masses. There's no intra or extrahepatic ductal dilatation. The gallbladder is slightly distended. The adrenal glands, pancreas, spleen, and left kidney are normal. A 12 millimeter simple cyst is present in.
Fused: administered. The lung bases are clear. The liver is enlarged and decreased in attenuation. There are no focal

# BONEYARD


In [171]:
# # =============================================================
# #  One-cell evaluation (uses your original jiwer transform)
# #  -------------------------------------------------------------
# #  Metrics per model:
# #    • Global WER                --> same as your old script
# #    • Medical-Term Recall (MTR) --> fraction of terms perfectly present
# #    • Medical-Term-only WER     --> WER on tokens that belong to terms
# #
# #  Expects a DataFrame `results_df` with columns:
# #        reference, vanilla, fused, medical_terms
# #  where medical_terms is list[str]  (or a string repr like "['a','b']").
# # =============================================================

# import re, ast, itertools, pandas as pd
# from jiwer import (
#     Compose, ToLowerCase, RemovePunctuation, RemoveMultipleSpaces,
#     Strip, ReduceToListOfListOfWords, wer
# )
# from unidecode import unidecode


# # ---------- helper to handle both str & list[str] -------------------
# def _map(func, x):
#     return [func(t) for t in x] if isinstance(x, list) else func(x)

# def remove_diacritics(x):
#     return _map(unidecode, x)

# def split_hyphens_and_slashes(x):
#     return _map(lambda t: re.sub(r"[-–—/]", " ", t), x)

# def normalize_nums(x):
#     return _map(lambda t: re.sub(r"(\d)[-–—-](\d)", r"\1-\2", t), x)

# # ---------- your original jiwer transform ---------------------------
# transform = Compose([
#     ToLowerCase(),
#     remove_diacritics,
#     split_hyphens_and_slashes,
#     normalize_nums,
#     RemovePunctuation(),
#     RemoveMultipleSpaces(),
#     Strip(),
#     ReduceToListOfListOfWords(),   # -> [["word", ...], ...]
# ])

# def compute_wer(ref, hyp):
#     return wer(
#         ref, hyp,
#         reference_transform=transform,
#         hypothesis_transform=transform,
#     )

# # ---------- lightweight normaliser for term metrics -----------------
# _punc_rx   = re.compile(r"[^\w\s]")
# _range_rx  = re.compile(r"(\d)[-–—-](\d)")
# _split_rx  = re.compile(r"[-–—/]")

# def _normalise(text: str) -> str:
#     text = unidecode(text.lower())
#     text = _range_rx.sub(r"\1-\2", text)
#     text = _split_rx.sub(" ", text)
#     text = _punc_rx.sub(" ", text)
#     return re.sub(r"\s+", " ", text).strip()

# def _term_recall(row, hyp_text):
#     hyp_norm = _normalise(hyp_text)
#     hits = sum(1 for t in row["medical_terms"] if _normalise(t) in hyp_norm)
#     return hits / len(row["medical_terms"])

# def _extract_term_tokens(row, text):
#     tokens = _normalise(text).split()
#     keep   = [False] * len(tokens)
#     for term in row["medical_terms"]:
#         ttoks = _normalise(term).split()
#         for i in range(len(tokens) - len(ttoks) + 1):
#             if tokens[i:i+len(ttoks)] == ttoks:
#                 for j in range(i, i+len(ttoks)):
#                     keep[j] = True
#     return " ".join(tok for tok, flag in zip(tokens, keep) if flag)

# # ---------- main evaluation routine ---------------------------------
# def evaluate(df: pd.DataFrame) -> None:
#     # ensure list[str] in medical_terms
#     if isinstance(df["medical_terms"].iloc[0], str):
#         df["medical_terms"] = df["medical_terms"].apply(ast.literal_eval)

#     # Global WER (your existing metric)
#     df["wer_vanilla"] = df.apply(
#         lambda r: compute_wer(r["reference"], r["vanilla"]), axis=1)
#     df["wer_fused"]   = df.apply(
#         lambda r: compute_wer(r["reference"], r["fused"]), axis=1)

#     # Medical-Term Recall
#     df["mtr_vanilla"] = df.apply(
#         lambda r: _term_recall(r, r["vanilla"]), axis=1)
#     df["mtr_fused"]   = df.apply(
#         lambda r: _term_recall(r, r["fused"]), axis=1)

#     # Medical-Term-only WER
#     df["mtwer_vanilla"] = df.apply(
#         lambda r: wer(
#             _extract_term_tokens(r, r["reference"]),
#             _extract_term_tokens(r, r["vanilla"]),
#             reference_transform=transform,
#             hypothesis_transform=transform), axis=1)
#     df["mtwer_fused"]   = df.apply(
#         lambda r: wer(
#             _extract_term_tokens(r, r["reference"]),
#             _extract_term_tokens(r, r["fused"]),
#             reference_transform=transform,
#             hypothesis_transform=transform), axis=1)

#     # -------- summary printout --------------------------------------
#     print("\n=== Global WER ===")
#     print(f"  vanilla : {df['wer_vanilla'].mean():.4f}")
#     print(f"  fused   : {df['wer_fused'].mean():.4f}")

#     print("\n=== Medical-Term Recall ===")
#     print(f"  vanilla : {df['mtr_vanilla'].mean():.4f}")
#     print(f"  fused   : {df['mtr_fused'].mean():.4f}")

#     print("\n=== Medical-Term-only WER ===")
#     print(f"  vanilla : {df['mtwer_vanilla'].mean():.4f}")
#     print(f"  fused   : {df['mtwer_fused'].mean():.4f}")

# # ---------- run on your DataFrame -----------------------------------

# import pandas as pd 

# results_df = pd.DataFrame(
#     {
#         "vanilla":[i.strip() for i in vanilla], 
#         "fused":[i.strip() for i in fused], 
#         "reference":ds['text'],
#         "medical_terms":ds['medical_terms']
#     }
# )

# evaluate(results_df.copy())

In [ ]:
# from transformers import LogitsProcessor, LogitsProcessorList
# import torch.nn.functional as F

# class ShallowFusion(LogitsProcessor):
#     def __init__(self, lm, shared_vocab, eos, alpha=0.3, warmup_steps=3):
#         super().__init__()
#         self.lm = lm.eval().requires_grad_(False)
#         self.V = shared_vocab
#         self.eos = eos
#         self.alpha = alpha
#         self.warmup = warmup_steps
#         self.step = 0

#     @torch.no_grad()
#     def __call__(self, input_ids, scores):
#         print('printing input_ids.size(), scores.size(), step, input_ids, dec_ids')
#         print(input_ids.size(), scores.size(), self.step, input_ids, processor.batch_decode(input_ids))
#         self.step+=1 

#         return scores
    
# fusion_proc = ShallowFusion(
#     lm=gpt2,
#     shared_vocab=gpt2.config.vocab_size,
#     eos=EOS_ID,
#     alpha=0.3
# )

# batch = next(iter(loader))
# feats = batch['input_features'].to(DEVICE)
# masks = batch['attention_mask'].to(DEVICE)

# with torch.no_grad():

#     out1 = whisper.generate(
#         input_features=feats,
#         attention_mask=masks,
#         logits_processor=LogitsProcessorList([fusion_proc]),
#         return_dict_in_generate=True,
#         output_scores=True,
#         num_beams=2,
#     )

#     # out2 = whisper.generate(
#     #     input_features=feats,
#     #     attention_mask=masks,
#     #     return_dict_in_generate=True,
#     #     output_scores=True,
#     #     num_beams=2
#     # )

In [ ]:
batch = next(iter(loader))
feats = batch['input_features'].to(DEVICE)
masks = batch['attention_mask'].to(DEVICE)

# Generate to get sequences
with torch.no_grad():
    out = whisper.generate(
        input_features=feats,
        attention_mask=masks,
        num_beams=1,
        do_sample=False,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=5
    )
# Compare step 0
decoder_ids = out.sequences[:,0:-1]  # Just the start token
with torch.no_grad():
    direct_logits = whisper(feats, decoder_input_ids=decoder_ids).logits[:, -1, :].to(DEVICE)
    direct_lp = torch.log_softmax(direct_logits, dim=-1)

gen_lp = out.scores[-1].to(DEVICE)

print(gen_lp)
print(direct_logits)


In [ ]:
decoder_ids

In [ ]:
# filter truly out of bounds vocab >=EOS
oob_mask = decoder_ids > EOS_ID # create mask for gpt2 OOV tokens emitted by whisper
 # replace with gpt2 pad token
filtered = decoder_ids.masked_fill(oob_mask, gpt2_tok.pad_token_id)
attention_mask = (filtered != gpt2_tok.pad_token_id).long()

with torch.no_grad():
    logits_new = gpt2(input_ids=filtered, attention_mask=attention_mask).logits[:,-1, :]

# because we dont want gpt2 to impact or determine termination just ASR model
logits_new[:,:EOS_ID-1].size()